## Financial Data Analysis Dashboard 

### Objective
Build a financial data analysis dashboard. The dashboard allows users to:
1. Filter transactions by bank.
2. Select specific date ranges or predefined periods (e.g., last week, last month).
3. Visualize financial data with bar and pie charts.
4. Display financial analysis insights.

In [1]:
import pandas as pd

df5 = pd.read_csv('Preprocessed_Data.csv')

### Install Dependencies

In [4]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install jupyter-dash

Note: you may need to restart the kernel to use updated packages.


To ensure that the transaction_date column in the dataset is properly formatted as a datetime object for easier manipulation and filtering.

In [9]:
df5['transaction_date'] = pd.to_datetime(df5['transaction_date'], format='%d-%m-%Y', errors='coerce')

### Financial Analysis Dashboard 

In [8]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import datetime

# Initialize the Dash app
app = Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
        id='bank-dropdown',
        options=[
           {'label': sender, 'value': sender} for sender in df5[df5['Category'] == 'Financial']['Sender'].unique()
        ],
        value=None,  # Default value
        placeholder="اختر البنك",  # Placeholder text
        clearable=True,
        style={
            'width': '200px',  # Adjust width
            'font-size': '12px',  # Adjust font size
            'padding': '5px',  # Adjust padding
            'text-align': 'center'  # Align placeholder text to center
        }
    ),
    dcc.Dropdown(
        id='period-dropdown',
        options=[
            {'label': 'آخر أسبوع', 'value': 'last_week'},
            {'label': 'آخر شهر', 'value': 'last_month'},
            {'label': 'حدد فترة', 'value': 'custom'}
        ],
        value=None,
        placeholder="تصفية تاريخ",
        clearable=False,
        style={
            'width': '150px',
            'font-size': '12px',
            'padding': '5px',
            'text-align': 'center'
        }
    ),
    html.Div(
        id='date-picker-container',
        children=[
            dcc.DatePickerRange(
                id='date-picker',
                start_date=df5['transaction_date'].min(),
                end_date=df5['transaction_date'].max(),
                display_format='YYYY-MM-DD'
            )
        ],
        style={'display': 'none'}
    ),
    dcc.Graph(id='bar-chart'),
    dcc.Graph(id='pie-chart'),
    html.Div(id='financial-analysis', style={'margin-top': '20px', 'font-size': '16px', 'line-height': '1.8em'})
])

# Callback to toggle the visibility of the date picker
@app.callback(
    Output('date-picker-container', 'style'),
    Input('period-dropdown', 'value')
)
def toggle_date_picker(period):
    if period == 'custom':
        return {'display': 'block'}  # Show date picker
    return {'display': 'none'}  # Hide date picker

# Callback to update the date picker values based on the selected period
@app.callback(
    [Output('date-picker', 'start_date'),
     Output('date-picker', 'end_date')],
    [Input('period-dropdown', 'value')]
)
def update_date_picker(period):
    today = datetime.date.today()
    if period == 'last_week':
        start_date = today - datetime.timedelta(weeks=1)
        end_date = today
    elif period == 'last_month':
        start_date = today - datetime.timedelta(days=30)
        end_date = today
    else:  # Custom Range
        start_date = df5['transaction_date'].min()
        end_date = df5['transaction_date'].max()
    return start_date, end_date

@app.callback(
    [Output('bar-chart', 'figure'),
     Output('pie-chart', 'figure'),
     Output('financial-analysis', 'children')],
    [Input('bank-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_charts_and_analysis(selected_bank, start_date, end_date):
    # Filter data by selected bank
    if selected_bank:
        filtered_df = df5[df5['Sender'] == selected_bank]
    else:
        filtered_df = df5

    # Further filter by date range
    filtered_df = filtered_df[(filtered_df['transaction_date'] >= start_date) & (filtered_df['transaction_date'] <= end_date)]

    # Calculate summary
    summary = filtered_df.groupby('transaction_type')['transaction_amount'].sum()
    income = summary.get('Income', 0)
    expense = summary.get('Expense', 0)
    savings = income - expense
    zero_savings = max(savings, 0)  # Ensure savings for pie chart are non-negative

    # Bar chart
    bar_fig = go.Figure()
    bar_fig.add_trace(go.Bar(
        x=['الدخل', 'المصروفات', 'المدخرات'],
        y=[income, expense, zero_savings],
        marker_color=['blue', 'orange', 'green']
    ))
    bar_fig.update_layout(template='plotly_white', height=500, width=700)

    # Pie chart
    labels = ['الدخل', 'المصروفات', 'المدخرات']
    values = [income, expense, zero_savings]
    filtered_labels = [label for label, value in zip(labels, values) if value > 0]
    filtered_values = [value for value in values if value > 0]

    pie_fig = go.Figure()
    pie_fig.add_trace(go.Pie(
        labels=filtered_labels,
        values=filtered_values,
        textinfo='none',
        hoverinfo='label+value',
        marker=dict(colors=['blue', 'orange', 'green'])
    ))
    pie_fig.update_layout(template='plotly_white', height=500, width=700)

    # Financial analysis text
    financial_analysis = html.Div([
        html.B("التحليل المالي", style={"font-size": "18px", "color": "#333"}),
        html.Br(),
        html.Span(f"إجمالي الدخل: {income} ريال سعودي", style={"font-size": "16px"}),
        html.Br(),
        html.Span(f"إجمالي المصروفات: {expense} ريال سعودي", style={"font-size": "16px"}),
        html.Br(),
        html.Span(f"صافي المدخرات: {zero_savings} ريال سعودي", style={"font-size": "16px"}),
        html.Br(),
        html.Span(
            "الملاحظة الرئيسية: ",
            style={"font-size": "16px", "font-weight": "bold"}
        ),
        html.Span(
            "صافي مدخراتك إيجابي، مما يشير إلى إدارة مالية جيدة. فكر في زيادة الاستثمارات للنمو المستقبلي."
            if savings > 0 else
            "صافي مدخراتك سلبي، مما يشير إلى زيادة المصروفات. راجع ميزانيتك للتحكم في النفقات."
            if savings < 0 else
            "صافي مدخراتك صفر، حاول تحسين التوازن بين الدخل والمصروفات.",
            style={"font-size": "16px"}
        )
    ], style={
        "line-height": "1.8em",
        "text-align": "right",  # Align text for Arabic
        "font-family": "'Cairo', sans-serif",  # Optional Arabic web font
        "color": "#333",  # Text color
        "margin-top": "20px"
    })

    return bar_fig, pie_fig, financial_analysis

if __name__ == '__main__':
    app.run_server(debug=True, port=8055)